In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
import hashlib
combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
hash_object = hashlib.md5(combined.encode())
hash_hex = hash_object.hexdigest()
hash_hex

'886d1617ec4af092246c7b5245234c9c'

In [73]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:20]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [74]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [75]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bf8fc38'}

In [76]:
from collections import defaultdict

In [77]:
hashes = defaultdict(list)
print(hashes)
for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)


defaultdict(<class 'list'>, {})


In [78]:
len(hashes), len(documents)

(947, 948)

In [79]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

4fcdc430 2


In [81]:
hashes['4fcdc430']

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '4fcdc430'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '4fcdc430'}]

In [66]:
import json
# json.loads: Deserializes a JSON string to a Python object.
# json.dumps: Serializes a Python object to a JSON string.

In [12]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [82]:
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


In [14]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [15]:
from openai import OpenAI
client = OpenAI()

In [16]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

## Testing Parts

In [44]:
doc=documents[0]
prompt = prompt_template.format(**doc)
print(prompt)
json_response=generate_questions(doc)
json.loads(json_response)


You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the course starts using this link.
Join the course Telegram channel with announcements.
Don’t forget to register in DataTalks.Club's Slack and join the channel.

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [32]:
from tqdm.auto import tqdm

In [33]:
results = {}

In [34]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|          | 0/948 [00:00<?, ?it/s]

In [20]:
import pickle

In [21]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [22]:
results['1f6520ca']

'["Where can I find the prerequisites for this course?", "How do I check the prerequisites for this course?", "Where are the course prerequisites listed?", "What are the requirements for joining this course?", "Where is the list of prerequisites for the course?"]'

In [36]:
try:
    # Attempt to decode the JSON
    data = json.loads(json_questions)
except json.JSONDecodeError as e:
    print(f"JSON decoding error: {e}")

JSON decoding error: Expecting ',' delimiter: line 1 column 311 (char 310)


In [64]:
parsed_resulst = {}

for doc_id, json_questions in results.items():
    #print(doc_id,json_questions)
    parsed_resulst[doc_id] = json.loads(json_questions)

JSONDecodeError: Invalid \escape: line 1 column 138 (char 137)

In [65]:
print(json_questions, doc_id)


["How can I resolve the error building Docker images on a Mac with an M1 chipset?", "What should I do if I encounter the 'Could not open \'/lib64/ld-linux-x86-64.so.2\'' error on my Mac?", "What specific line should be changed in the Dockerfile to fix Docker image build errors on Mac M1?", "Where can I find the Dockerfile that needs modification to resolve the Docker image build issue on Mac M1?", "How long did it take to build the Docker image after fixing the issue on the Mac M1?"] ec88d101


In [ ]:
json_questions=[
  "What should I do if I get a 'ModuleNotFoundError: No module named \"kafka.vendor.six.moves\"?'",
  "Which library should I use to resolve the issue with 'kafka.vendor.six.moves'?",
  "Where can I find the recommended fix for the kafka-python module error?",
  "What is the suggested command to install the alternative kafka-python library?",
  "Why should I use kafka-python-ng instead of the original kafka-python?"
]

In [63]:
results[doc_id]=json.dumps(json_questions)

In [ ]:
doc_index = {d['id']: d for d in documents}

In [ ]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [ ]:
df.to_csv('ground-truth-data.csv', index=False)

In [ ]:
!head ground-truth-data.csv